In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
import os
import glob as gb
import cv2
import tensorflow as tf
import keras

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3744\2879806208.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [4]:
trainpath="C:\\Users\\Lenovo\\Desktop\\emo\\train\\"
testpath="C:\\Users\\Lenovo\\Desktop\\emo\\test\\"

In [5]:
for folder in  os.listdir(trainpath + 'train') : 
    files = gb.glob(pathname= str( trainpath +'train//' + folder + '/*.jpg'))
    print(f'For training data , found {len(files)} in folder {folder}')

For training data , found 16 in folder Anger
For training data , found 16 in folder Contempt
For training data , found 16 in folder Disgust
For training data , found 16 in folder Fear
For training data , found 16 in folder Happy
For training data , found 16 in folder Neutral
For training data , found 16 in folder Sad
For training data , found 16 in folder Surprised


In [6]:
code = {'Anger':0 ,'Contempt':1,'Fear':2,'Disgust':3,'Happy':4,'Neutral':5,'Sad':6,'Surprised':7}

def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x 

In [7]:
size = []
for folder in  os.listdir(trainpath +'train') : 
    files = gb.glob(pathname= str( trainpath +'train//' + folder + '/*.jpg'))
    for file in files: 
        image = plt.imread(file)
        size.append(image.shape)
pd.Series(size).value_counts()

(3264, 2448, 3)    34
(2592, 1944, 3)    19
(720, 1280, 3)     13
(3888, 3888, 3)     8
(4608, 2592, 3)     8
(3216, 1448, 3)     8
(3280, 2464, 3)     7
(1120, 2304, 3)     7
(5184, 3880, 3)     7
(3408, 2556, 3)     6
(3264, 1472, 3)     6
(4608, 3456, 3)     5
Name: count, dtype: int64

In [17]:
s=500

In [18]:
X_train = []
y_train = []
for folder in  os.listdir(trainpath +'train') : 
    files = gb.glob(pathname= str( trainpath +'train//' + folder + '/*.jpg'))
    for file in files: 
        image = cv2.imread(file)
        image_array = cv2.resize(image , (s,s))
        X_train.append(list(image_array))
        y_train.append(code[folder])


In [19]:
print(f'we have {len(X_train)} items in X_train')

we have 128 items in X_train


In [20]:
print(f'we have {len(y_train)} items in y_train')

we have 128 items in y_train


In [21]:
X_test = []
y_test = []
for folder in  os.listdir(testpath +'test') : 
    files = gb.glob(pathname= str(testpath + 'test//' + folder + '/*.jpg'))
    for file in files: 
        image = cv2.imread(file)
        image_array = cv2.resize(image , (s,s))
        X_test.append(list(image_array))
        y_test.append(code[folder])
        

In [22]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(f'X_train shape  is {X_train.shape}')
print(f'X_test shape  is {X_test.shape}')
print(f'y_train shape  is {y_train.shape}')
print(f'y_test shape  is {y_test.shape}')

X_train shape  is (128, 500, 500, 3)
X_test shape  is (24, 500, 500, 3)
y_train shape  is (128,)
y_test shape  is (24,)


In [23]:
KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(128,kernel_size=(3,3),activation='relu',input_shape=(s,s,3)),
        keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'),    
        keras.layers.MaxPool2D(4,4),
        keras.layers.Flatten() ,    
        keras.layers.Dense(64,activation='relu') ,    
        keras.layers.Dense(32,activation='relu') ,    
        keras.layers.Dropout(rate=0.5) ,            
        keras.layers.Dense(8,activation='softmax') ,    
        ])

In [24]:
KerasModel.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [25]:
print('Model Details are : ')
print(KerasModel.summary())

Model Details are : 
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 498, 498, 128)     3584      
                                                                 
 conv2d_4 (Conv2D)           (None, 496, 496, 64)      73792     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 124, 124, 64)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 122, 122, 64)      36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (Non

In [ ]:
epochs = 5
ThisModel = KerasModel.fit(X_train, y_train, epochs=epochs,batch_size=32,verbose=1)

Epoch 1/5
1/4 [======>.......................] - ETA: 8:28 - loss: 30.3608 - accuracy: 0.0938